# Benchmark de Prétraitement de Texte

Ce notebook démontre les résultats de l'étude de benchmark pour identifier la meilleure stratégie de nettoyage de texte pour la classification de produits Rakuten.

## Objectifs

1. Comparer différentes stratégies de prétraitement de texte
2. Identifier la configuration optimale pour la classification
3. Mesurer l'impact de chaque étape de nettoyage

## Méthodologie

- **Modèle** : Pipeline TF-IDF + Régression Logistique
- **Métrique** : F1 Score (pondéré)
- **Données** : 84,916 produits Rakuten avec 27 catégories
- **Division** : 80% train / 20% test

---

## 1. Importations et Configuration

In [4]:
import sys
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

sys.path.insert(0, '../src')

# Importer les modules de prétraitement et benchmark
from rakuten_text.preprocessing import clean_text, final_text_cleaner, print_available_options
from rakuten_text.benchmark import load_dataset, define_experiments, run_benchmark, analyze_results, save_results

## 2. Chargement des Données

In [5]:
# Charger le dataset Rakuten
df = load_dataset(data_dir="../data/raw")

## 3. Exploration des Options de Nettoyage

Voyons toutes les options de nettoyage disponibles :

In [6]:
# Afficher toutes les options de nettoyage disponibles
print_available_options()

Options de nettoyage disponibles :
  fix_encoding              : Corriger l'encodage de texte cassé avec ftfy
  unescape_html             : Décoder les entités HTML (&amp; → &)
  normalize_unicode         : Appliquer la normalisation Unicode NFC
  remove_html_tags          : Supprimer les balises HTML <tag>contenu</tag>
  remove_boilerplate        : Supprimer les phrases de template communes
  lowercase                 : Convertir en minuscules
  merge_dimensions          : Fusionner les motifs de dimensions (22 x 11 → 22x11)
  merge_units               : Fusionner les unités numériques (500 g → 500g)
  merge_durations           : Fusionner les durées (24 h → 24h)
  merge_age_ranges          : Fusionner les tranches d'âge (3-5 ans → 3_5ans)
  tag_years                 : Étiqueter les années à 4 chiffres (1917 → year1917)
  remove_punctuation        : Supprimer les signes de ponctuation isolés
  remove_stopwords          : Supprimer les mots vides français/anglais
  remove_single_letter

## 4. Benchmark Complet

Exécutons le benchmark complet pour comparer toutes les stratégies de nettoyage.

**Note** : Cette cellule peut prendre plusieurs minutes à s'exécuter.

In [7]:
# Définir les expériences
experiments = define_experiments()

print(f"Nombre total d'expériences : {len(experiments)}")
print("\nExemples d'expériences :")
for exp in experiments[:5]:
    print(f"  - {exp['name']:30s} (Groupe: {exp['group']})")
print("  ...")

Nombre total d'expériences : 22

Exemples d'expériences :
  - baseline_raw                   (Groupe: 0_Baseline)
  - fix_encoding                   (Groupe: 1_Encodage)
  - unescape_html                  (Groupe: 1_Encodage)
  - normalize_unicode              (Groupe: 1_Encodage)
  - all_encoding_fixes             (Groupe: 1_Encodage)
  ...


In [8]:
# Exécuter le benchmark

results_df = run_benchmark(
    df=df,
    experiments=experiments,
    test_size=0.2,
    random_state=42,
    tfidf_max_features=10000,
    tfidf_ngram_range=(1, 2),
    verbose=True
)

CONFIGURATION DU BENCHMARK
Total expériences      : 22
Taille de test         : 0.2
État aléatoire         : 42
TF-IDF max features    : 10,000
TF-IDF plage n-grammes : (1, 2)

Création de la division train/test...
  Train : 67,932 échantillons
  Test  : 16,984 échantillons

[1/22] baseline_raw
  Groupe : 0_Baseline
  Config : Aucune (données brutes)
  Nettoyage du texte... ✓ (longueur moyenne : 595 caractères)
  Entraînement... ✓
  Évaluation... ✓
  → Score F1 : 0.791787 | Exactitude : 0.7915 | [BASELINE]

[2/22] fix_encoding
  Groupe : 1_Encodage
  Config : {'fix_encoding': True}
  Nettoyage du texte... ✓ (longueur moyenne : 588 caractères)
  Entraînement... ✓
  Évaluation... ✓
  → Score F1 : 0.792359 | Exactitude : 0.7921 | Δ vs baseline : 🚀 +0.000572 (+0.07%)

[3/22] unescape_html
  Groupe : 1_Encodage
  Config : {'unescape_html': True}
  Nettoyage du texte... ✓ (longueur moyenne : 584 caractères)
  Entraînement... ✓
  Évaluation... ✓
  → Score F1 : 0.792628 | Exactitude : 0.7924 |

## 5. Analyse des Résultats

In [9]:
# Analyser les résultats
analyze_results(results_df, top_n=10)


ANALYSE DES RÉSULTATS DU BENCHMARK

Score F1 Baseline : 0.791787

🚀 TOP 10 AMÉLIORATIONS :
--------------------------------------------------------------------------------
  traditional_cleaning           | F1 : 0.802510 | Δ : +0.010723 (+1.35%) | Groupe : 7_Combos
  lowercase                      | F1 : 0.796803 | Δ : +0.005016 (+0.63%) | Groupe : 3_Casse
  remove_stopwords               | F1 : 0.794423 | Δ : +0.002636 (+0.33%) | Groupe : 6_Filtrage
  remove_html_tags               | F1 : 0.793315 | Δ : +0.001528 (+0.19%) | Groupe : 2_HTML
  merge_dimensions               | F1 : 0.792743 | Δ : +0.000956 (+0.12%) | Groupe : 4_Fusions
  merge_durations                | F1 : 0.792710 | Δ : +0.000923 (+0.12%) | Groupe : 4_Fusions
  unescape_html                  | F1 : 0.792628 | Δ : +0.000842 (+0.11%) | Groupe : 1_Encodage
  conservative_cleaning          | F1 : 0.792392 | Δ : +0.000605 (+0.08%) | Groupe : 7_Combos
  merge_age_ranges               | F1 : 0.792359 | Δ : +0.000572 (+0.07%